Columns to keep!
- Rndrng_Prvdr_State_Abrvtn (Provider State)
- DRG_Desc (DRG Definition)
- Tot_Dschrgs (Total Discharges)
- Avg_Submtd_Cvrd_Chrg	(Average Covered Charges)
- Avg_Tot_Pymt_Amt (Average Total Payments)
- Avg_Mdcr_Pymt_Amt (Average payment from Medicare)




In [1]:
import pandas as pd

columns_to_keep = [
    "Rndrng_Prvdr_State_Abrvtn",
    "DRG_Desc",
    "Tot_Dschrgs",
    "Avg_Submtd_Cvrd_Chrg",
    "Avg_Tot_Pymt_Amt",
    "Avg_Mdcr_Pymt_Amt"
]

cms = pd.read_csv(
    "data/inpatient.csv",
    usecols=columns_to_keep,
    encoding="windows-1252",
    low_memory=False
)

print(cms.shape)
cms.head(5)


(146427, 6)


,Rndrng_Prvdr_State_Abrvtn,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,AL,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,14,663764.35714,120219.928570,115544.142860
1,AL,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,26,180980.88462,37321.038462,35261.807692
2,AL,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,12,105824.33333,26936.666667,25048.916667
3,AL,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCE...,16,242539.50000,34745.375000,32438.625000
4,AL,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.363636


In [2]:
cms = cms.rename(columns={
    "Rndrng_Prvdr_State_Abrvtn": "Provider_State",
    "DRG_Desc": "DRG_Def",
    "Avg_Submtd_Cvrd_Chrg": "Avg_covered_charges",
    "Avg_Tot_Pymt_Amt": "Avg_total_payments",
    "Tot_Dschrgs": "Total_Discharges",
    "Avg_Mdcr_Pymt_Amt":"Avg_pay_from_medicare"
})
cms.head(5)


,Provider_State,DRG_Def,Total_Discharges,Avg_covered_charges,Avg_total_payments,Avg_pay_from_medicare
0,AL,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,14,663764.35714,120219.928570,115544.142860
1,AL,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,26,180980.88462,37321.038462,35261.807692
2,AL,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,12,105824.33333,26936.666667,25048.916667
3,AL,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCE...,16,242539.50000,34745.375000,32438.625000
4,AL,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.363636


In [3]:
cms.isnull().sum()


Provider_State           0
DRG_Def                  0
Total_Discharges         0
Avg_covered_charges      0
Avg_total_payments       0
Avg_pay_from_medicare    0
dtype: int64

In [4]:
cdi =  pd.read_csv("data/Chronic_Disease.csv")
cdi.head(5)

/var/folders/5_/7850fm454vb567dm0m8jx2d80000gn/T/ipykernel_41823/3157787753.py:1: DtypeWarning: Columns (10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  cdi =  pd.read_csv("data/Chronic_Disease.csv")


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2020,2020,US,United States,BRFSS,Health Status,Recent activity limitation among adults,NaN,Number,Age-adjusted Mean,...,HEA,HEA04,NaN,AGEADJMEAN,SEX,SEXF,NaN,NaN,NaN,NaN
1,2015,2019,AR,Arkansas,US Cancer DVT,Cancer,"Invasive cancer (all sites combined), incidence",NaN,Number,Number,...,CAN,CAN07,NaN,NMBR,SEX,SEXM,NaN,NaN,NaN,NaN
2,2015,2019,CA,California,US Cancer DVT,Cancer,"Cervical cancer mortality among all females, u...",NaN,Number,Number,...,CAN,CAN03,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2015,2019,CO,Colorado,US Cancer DVT,Cancer,"Invasive cancer (all sites combined), incidence",NaN,Number,Number,...,CAN,CAN07,NaN,NMBR,RACE,HIS,NaN,NaN,NaN,NaN
4,2015,2019,GA,Georgia,US Cancer DVT,Cancer,"Prostate cancer mortality among all males, und...",NaN,Number,Number,...,CAN,CAN05,NaN,NMBR,RACE,WHT,NaN,NaN,NaN,NaN


columns_to_keep 
- "LocationAbbr"
-  "Topic" 
-  "Question"
-   "DataValue"
-    "YearStart"


In [ ]:
import pandas as pd

# Filter for key chronic disease topics
topics_to_keep = ["Diabetes", "Cardiovascular Disease", "COPD", "Cancer", "Obesity"]

cdi = cdi[cdi["Topic"].isin(topics_to_keep)]

columns_to_keep = ["LocationDesc", "Topic", "Question", "DataValue", "YearStart"]
cdi = cdi[columns_to_keep]



(70385, 5)


,LocationDesc,Topic,Question,DataValue,YearStart
1,Arkansas,Cancer,"Invasive cancer (all sites combined), incidence","9,537",2015
2,California,Cancer,"Cervical cancer mortality among all females, u...",486,2015
3,Colorado,Cancer,"Invasive cancer (all sites combined), incidence","2,880",2015
4,Georgia,Cancer,"Prostate cancer mortality among all males, und...",519,2015
5,Kansas,Cancer,"Invasive cancer (all sites combined), incidence","8,102",2015


In [7]:
cdi =  cdi.rename(columns = {
    "LocationDesc" : "State",
    "DataValue": "Value",
    "YearStart":"Year" 
})
cdi.head()


,State,Topic,Question,Value,Year
1,Arkansas,Cancer,"Invasive cancer (all sites combined), incidence","9,537",2015
2,California,Cancer,"Cervical cancer mortality among all females, u...",486,2015
3,Colorado,Cancer,"Invasive cancer (all sites combined), incidence","2,880",2015
4,Georgia,Cancer,"Prostate cancer mortality among all males, und...",519,2015
5,Kansas,Cancer,"Invasive cancer (all sites combined), incidence","8,102",2015


In [ ]:
cdi.isnull().sum()

#Dropping the null values
cdi = cdi.dropna(subset=["Value"])r
